# Imports and Helper Functions

In [ ]:
import sys
sys.path.append('../')
from inference import *
import glob, os
DATA_PATH = '../simulation/data/test_data/'

# Generate Predictions (if not already generated)

In [ ]:
# Load Model
ckpt_dir = '../core/static/weights/strategy1_run2_v2_pgn/'
# ckpt_dir = '../core/static/weights/strategy2_run3_v2_pgn/'
model, params = load_model(ckpt_dir)

# Aggregate Data
data = glob.glob(os.path.join(DATA_PATH, '*.parquet'))

In [ ]:
# Print best train and validation loss
print('Best Train Loss: ', params['history']['train_loss'][-1])
print('Best Validation Loss: ', params['history']['val_loss'][-1])

# Inference (if not already done)
if not os.path.isfile('./preds_strat1.pkl' if params['model']['strategy'] == 1 else './preds_strat2.pkl'):
    preds = []
    print("Performing inference...")
    for d in tqdm(data, leave=True, unit="file"):
        d = load_points(d, verbose=False)
        dfs = []
        for g in tqdm(d, leave=False, unit="batch"):
            df = infer(
                model, g, params["model"]["strategy"], NOISE_THRESH, DEVICE
            )
            dfs.append(df)

        df = pd.concat(dfs)

        # # Get rid of nans
        # df.dropna(inplace=True)

        # Drop duplicates (if any)
        df = df.drop_duplicates(subset=["x", "y", "z"], keep="first")

        # Save
        preds.append(df)
    print("Done!")

In [ ]:
# Save Predictions
import pickle

if not os.path.isfile('./preds_strat1.pkl' if params['model']['strategy'] == 1 else './preds_strat2.pkl'):
    with open('preds_strat1.pkl' if params["model"]["strategy"] == 1 else 'preds_strat2.pkl', 'wb') as f:
        pickle.dump(preds, f)
else:
    preds = pickle.load(open('preds_strat1.pkl', 'rb')) if params["model"]["strategy"] == 1 else pickle.load(open('preds_strat2.pkl', 'rb'))

# Metric Calculation

In [ ]:
# Load all the point clouds
gts = [pd.read_parquet(d) for d in data]
sum([len(g) for g in gts]), sum([len(p) for p in preds])

In [ ]:
is_noise_preds = [df["label"] == NOISE_LABEL for df in preds]
is_noise_gts = [df["label"] == 'noise_bg' for df in gts]

# true positive, false positive, true negative, false negative
tp = [np.sum(p & g) for p, g in zip(is_noise_preds, is_noise_gts)]
fp = [np.sum(p & ~g) for p, g in zip(is_noise_preds, is_noise_gts)]
tn = [np.sum(~p & ~g) for p, g in zip(is_noise_preds, is_noise_gts)]
fn = [np.sum(~p & g) for p, g in zip(is_noise_preds, is_noise_gts)]

# Noise prediction precision
noise_prec = np.sum(tp) / (np.sum(tp) + np.sum(fp))
print('Noise Prediction Precision: ', noise_prec)

# Noise prediction recall
noise_rec = np.sum(tp) / (np.sum(tp) + np.sum(fn))
print('Noise Prediction Recall: ', noise_rec)

# Noise prediction F1 score
noise_f1 = 2 * np.sum(tp) / (2 * np.sum(tp) + np.sum(fp) + np.sum(fn))
print('Noise Prediction F1 Score: ', noise_f1)

# Noise prediction accuracy
noise_acc = np.mean([np.mean(p.reset_index(drop=True) == g.reset_index(drop=True)) for p, g in zip(is_noise_preds, is_noise_gts)])
print('Noise Prediction Accuracy: ', noise_acc)


In [ ]:
angle_errors_noise_sensitive = []
angle_errors_noise_agnostic = []
for i in range(len(preds)):
    noise_pred = is_noise_preds[i]
    noise_gt = is_noise_gts[i]
    mask = ~noise_pred & ~noise_gt
    ang = torch.nn.functional.cosine_similarity(
        torch.tensor(preds[i][['nx', 'ny', 'nz']].values[mask]),
        torch.tensor(gts[i][['nx', 'ny', 'nz']].values[mask]), dim=1).cpu().numpy()
    error = np.arccos(pd.DataFrame(ang).mean()) * 180 / np.pi

    # Count dissimilarities in noise_pred and noise_gt, and give them a 90 degree error
    count = np.sum(np.logical_xor(noise_pred, noise_gt))
    final_error = (error * sum(mask) + count * 90) / (sum(mask) + count)
    
    angle_errors_noise_sensitive.append(final_error)  # Penalizes noise points
    angle_errors_noise_agnostic.append(error)  # Ignores noise points



# Angle error in degrees, between the ground truth normal and the predicted normal
print('Noise-sensitive angle error: ', np.mean(angle_errors_noise_sensitive))
print('Noise-agnostic angle error: ', np.mean(angle_errors_noise_agnostic))

# Print all the metric in a table

In [ ]:
metrics_df = pd.DataFrame(
    [[noise_prec, noise_rec, noise_f1, noise_acc, np.mean(angle_errors_noise_sensitive), np.mean(angle_errors_noise_agnostic)]],
    columns=['Noise Precision', 'Noise Recall', 'Noise F1', 'Noise Accuracy', 'Noise-sensitive Angle Error', 'Noise-agnostic Angle Error'])

metrics_df.T

In [ ]:
metrics_df.T.to_csv('metrics_strat1.csv' if params["model"]["strategy"] == 1 else 'metrics_strat2.csv')

In [ ]:
params['history']['best_epoch']